# WMT - $\theta$ budget using xhistogram

In [ ]:
import dask
import dask.array as dsa
import gcsfs
import gsw
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import xarray as xr
#import cartopy.crs as ccrs #cartopy coord ref sys
#import cartopy
#import cartopy.feature as cfeature

In [ ]:
#loading flux ds
ds_tmp = xr.open_zarr(gcsfs.GCSMap('pangeo-tmp/stb2145/SOSE/budget_th.zarr'))

#loading surface term ds
ds_tmp_surf_terms = xr.open_zarr(gcsfs.GCSMap('pangeo-tmp/stb2145/SOSE/budget_surf_th.zarr'))

#loading theta and salt ds
t_s = xr.open_zarr(gcsfs.GCSMap('pangeo-tmp/stb2145/SOSE/t_s.zarr'))

In [ ]:
#merging datasets to be organized
ds = xr.merge([ds_tmp, ds_tmp_surf_terms, t_s])
ds

In [ ]:
#rechunking by `chunksize=ds.shape` of each variable so xhistogram can be happy
ds = ds.chunk({'XC': -1, 'YC': -1, 'Z': -1})
ds

## Tracer Budgets

Here we will do the heat and salt budgets for SOSE. In integral form, these budgets can be written as

$$
\mathcal{V} \frac{\partial S}{\partial t} = G^S_{adv} + G^S_{diff} + G^S_{surf} + G^S_{linfs}
$$


$$
\mathcal{V} \frac{\partial \theta}{\partial t} = G^\theta_{adv} + G^\theta_{diff} + G^\theta_{surf} + G^\theta_{linfs} + G^\theta_{sw}
$$

where $\mathcal{V}$ is the volume of the grid cell. The terms on the right-hand side are called _tendencies_. They add up to the total tendency (the left hand side).

The first term is the convergence of advective fluxes. The second is the convergence of diffusive fluxes. The third is the explicit surface flux. The fourth is the correction due to the linear free-surface approximation. The fifth is shortwave penetration (only for temperature).

### Flux Divergence

First we define a function to calculate the convergence of the advective and diffusive fluxes, since this has to be repeated for both tracers.

**Do I need to `grid.diff(ds.THETA)`? I don't think so because we don't want to find the difference between each cell point for temperature right? Unless... we want to do that to make theta on the same coord as all the other fluxes**

In [ ]:
from xhistogram.xarray import histogram
#histogram?

In [ ]:
theta_bins = np.arange(-2, 30, 0.5)
theta_bins

In [ ]:
def hist_var(direction, tendency, suffix):
    """Compute histogram of `tendency` flux of tracer `suffix`
    where `tendency` is `ADV` or `DIFF`, `suffix` is `TH` or `SLT`,
    and `direction` is `horiz` or `vert`. Return a new xarray.Dataset."""
    
    var = 'conv_' + direction + '_' + tendency + '_flux_' + suffix
    hist_var = histogram(ds.THETA, bins=[theta_bins], weights=ds[var], dim=['XC', 'YC', 'Z'])
    return hist_var

In [ ]:
adv_vert_hist = hist_var('vert', 'adv', 'TH').load()
adv_horiz_hist = hist_var('horiz', 'adv', 'TH').load()
diff_vert_hist = hist_var('vert', 'diff', 'TH').load()
diff_horiz_hist = hist_var('horiz', 'diff', 'TH').load()

In [ ]:
surf_flux_hist = histogram(ds.THETA, bins=[theta_bins], weights=ds.surface_flux_conv_TH, dim=['XC', 'YC', 'Z']).load()
lin_fs_hist = histogram(ds.THETA, bins=[theta_bins], weights=ds.lin_fs_correction_TH, dim=['XC', 'YC', 'Z']).load()
sw_hist = histogram(ds.THETA, bins=[theta_bins], weights=ds.sw_flux_conv_TH, dim=['XC', 'YC', 'Z']).load()

In [ ]:
ds_hist = xr.merge([adv_horiz_hist.rename('adv_horiz_hist'), 
                    adv_vert_hist.rename('adv_vert_hist'),
                    diff_horiz_hist.rename('diff_horiz_hist'),
                    diff_vert_hist.rename('diff_vert_hist'),
                    surf_flux_hist.rename('surf_flux_hist'),
                    lin_fs_hist.rename('lin_fs_hist'),
                    sw_hist.rename('sw_hist')])
ds_hist

**How do I sum the variables in a dataset together and return a data array with same dimensions?**

In [ ]:
total_tendency_hist = (ds_hist.adv_horiz_hist + ds_hist.adv_vert_hist + 
                       ds_hist.diff_horiz_hist + ds_hist.diff_vert_hist +
                       ds_hist.surf_flux_hist + ds_hist.lin_fs_hist + 
                       ds_hist.sw_hist)

In [ ]:
ds_hist['total_tendency_hist'] = total_tendency_hist

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, sharex=True, figsize=(14,14))

ds_hist.adv_horiz_hist.transpose().plot(ax=axes[0,0], cmap='seismic')
ds_hist.diff_horiz_hist.transpose().plot(ax=axes[0,1], cmap='seismic')
ds_hist.adv_vert_hist.transpose().plot(ax=axes[1,0], cmap='seismic')
ds_hist.diff_vert_hist.transpose().plot(ax=axes[1,1], cmap='seismic')
ds_hist.surf_flux_hist.transpose().plot(ax=axes[2,0], cmap='seismic')
ds_hist.lin_fs_hist.transpose().plot(ax=axes[2,1], cmap='seismic')
ds_hist.sw_hist.transpose().plot(ax=axes[3,0], cmap='seismic')
ds_hist.total_tendency_hist.transpose().plot(ax=axes[3,1], cmap='seismic')

fig.suptitle('Tendency terms binned', fontsize=18, fontweight='bold')

axes[0,0].title.set_text('Horizontal Advective Flux Term')
axes[0,1].title.set_text('Horizontal Diffusive Flux Term')
axes[1,0].title.set_text('Vertical Advective Flux Term')
axes[1,1].title.set_text('Vertical Diffusive Flux Term')
axes[2,0].title.set_text('Surface Flux Term')
axes[2,1].title.set_text('Linear free-surface correction Term')
axes[3,0].title.set_text('Shortwave Term')
axes[3,1].title.set_text('Total Tendency Term')

plt.show() #this seems to work to make suptitle displayed neatly above
plt.tight_layout();

#### Notes:
- red colors/positive values represent temperature getting warmer and negative (blue) values represent temperature getting cooler (refer to figure Ryan drew on board and you copied down in 12/16/19 notes). To see if the temperature-watermass is growing in volume or not is to look at total tendency term. Individual terms won't tell you if watermass is growing or not, only their respective influence on its temperature profile.
- easier to look at time average first then look at temporal variability
- add air-sea flux (lin_fs_correction, surface_terms, sw_term)
- **Don't forget about ECCO!**

## Entire WG column budget

In [ ]:
lower_lat = 0
upper_lat = 78
left_lon = 1788
right_lon = 2159

In [ ]:
#set the boundaries
lat_range = dict(YC=slice(lower_lat, upper_lat))
lon_range = dict(XC=slice(left_lon, right_lon))

In [ ]:
ds_wg = ds.isel(**lat_range, **lon_range)
ds_wg

In [ ]:
def hist_var(direction, tendency, suffix):
    """Compute histogram of `tendency` flux of tracer `suffix`
    where `tendency` is `ADV` or `DIFF`, `suffix` is `TH` or `SLT`,
    and `direction` is `horiz` or `vert`. Return a new xarray.Dataset."""
    
    var = 'conv_' + direction + '_' + tendency + '_flux_' + suffix
    hist_var = histogram(ds_wg.THETA, bins=[theta_bins], weights=ds_wg[var], 
                         dim=['XC', 'YC', 'Z'])
    return hist_var

In [ ]:
adv_vert_hist_wg = hist_var('vert', 'adv', 'TH').load()
adv_horiz_hist_wg = hist_var('horiz', 'adv', 'TH').load()
diff_vert_hist_wg = hist_var('vert', 'diff', 'TH').load()
diff_horiz_hist_wg = hist_var('horiz', 'diff', 'TH').load()

In [ ]:
surf_flux_hist_wg = histogram(ds_wg.THETA, bins=[theta_bins], 
                              weights=ds_wg.surface_flux_conv_TH, dim=['XC', 'YC', 'Z']).load()
lin_fs_hist_wg = histogram(ds_wg.THETA, bins=[theta_bins], 
                           weights=ds_wg.lin_fs_correction_TH, dim=['XC', 'YC', 'Z']).load()
sw_hist_wg = histogram(ds_wg.THETA, bins=[theta_bins], 
                       weights=ds_wg.sw_flux_conv_TH, dim=['XC', 'YC', 'Z']).load()

In [ ]:
tot_tend_truth_hist_wg = histogram(ds_wg.THETA, bins=[theta_bins],
                                   weights=ds_wg.total_tendency_TH_truth, dim=['XC', 'YC', 'Z']).load()

In [ ]:
ds_hist_wg = xr.merge([adv_horiz_hist_wg.rename('adv_horiz_hist_wg'), 
                       adv_vert_hist_wg.rename('adv_vert_hist_wg'),
                       diff_horiz_hist_wg.rename('diff_horiz_hist_wg'),
                       diff_vert_hist_wg.rename('diff_vert_hist_wg'),
                       surf_flux_hist_wg.rename('surf_flux_hist_wg'),
                       lin_fs_hist_wg.rename('lin_fs_hist_wg'),
                       sw_hist_wg.rename('sw_hist_wg'),
                       tot_tend_truth_hist_wg.rename('tot_tend_truth_hist_wg')])
ds_hist_wg

In [ ]:
tot_tend_hist_wg = (ds_hist_wg.adv_horiz_hist_wg + ds_hist_wg.adv_vert_hist_wg + 
                    ds_hist_wg.diff_horiz_hist_wg + ds_hist_wg.diff_vert_hist_wg +
                    ds_hist_wg.surf_flux_hist_wg + ds_hist_wg.lin_fs_hist_wg + 
                    ds_hist_wg.sw_hist_wg)

In [ ]:
ds_hist_wg['tot_tend_hist_wg'] = tot_tend_hist_wg

In [ ]:
ds_hist_wg

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, sharex=True, figsize=(14,14))

ds_hist_wg.adv_horiz_hist_wg.transpose().plot(ax=axes[0,0], cmap='seismic')
ds_hist_wg.diff_horiz_hist_wg.transpose().plot(ax=axes[0,1], cmap='seismic')
ds_hist_wg.adv_vert_hist_wg.transpose().plot(ax=axes[1,0], cmap='seismic')
ds_hist_wg.diff_vert_hist_wg.transpose().plot(ax=axes[1,1], cmap='seismic')
ds_hist_wg.surf_flux_hist_wg.transpose().plot(ax=axes[2,0], cmap='seismic')
ds_hist_wg.lin_fs_hist_wg.transpose().plot(ax=axes[2,1], cmap='seismic')
ds_hist_wg.sw_hist_wg.transpose().plot(ax=axes[3,0], cmap='seismic')
ds_hist_wg.tot_tend_hist_wg.transpose().plot(ax=axes[3,1], cmap='seismic')

axes[0,0].set_ylim(-2, 5)
axes[0,1].set_ylim(-2, 5)
axes[1,0].set_ylim(-2, 5)
axes[1,1].set_ylim(-2, 5)
axes[2,0].set_ylim(-2, 5)
axes[2,1].set_ylim(-2, 5)
axes[3,0].set_ylim(-2, 5)
axes[3,1].set_ylim(-2, 5)

fig.suptitle('Tendency terms binned in WG region', fontsize=18, fontweight='bold')

axes[0,0].title.set_text('Horizontal Advective Flux Term')
axes[0,1].title.set_text('Horizontal Diffusive Flux Term')
axes[1,0].title.set_text('Vertical Advective Flux Term')
axes[1,1].title.set_text('Vertical Diffusive Flux Term')
axes[2,0].title.set_text('Surface Flux Term')
axes[2,1].title.set_text('Linear free-surface correction Term')
axes[3,0].title.set_text('Shortwave Term')
axes[3,1].title.set_text('Total Tendency Term')

plt.show() #this seems to work to make suptitle displayed neatly above
plt.tight_layout();

#### Time-Avged

In [ ]:
ds_hist_wg_mean = ds_hist_wg.mean(dim='time')
ds_hist_wg_mean

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, sharex=True, figsize=(14,14))

ds_hist_wg_mean.adv_horiz_hist_wg.transpose().plot(ax=axes[0,0])#, cmap='seismic')
ds_hist_wg_mean.diff_horiz_hist_wg.transpose().plot(ax=axes[0,1])#, cmap='seismic')
ds_hist_wg_mean.adv_vert_hist_wg.transpose().plot(ax=axes[1,0])#, cmap='seismic')
ds_hist_wg_mean.diff_vert_hist_wg.transpose().plot(ax=axes[1,1])#, cmap='seismic')
ds_hist_wg_mean.surf_flux_hist_wg.transpose().plot(ax=axes[2,0])#, cmap='seismic')
ds_hist_wg_mean.lin_fs_hist_wg.transpose().plot(ax=axes[2,1])#, cmap='seismic')
ds_hist_wg_mean.sw_hist_wg.transpose().plot(ax=axes[3,0])#, cmap='seismic')
ds_hist_wg_mean.tot_tend_hist_wg.transpose().plot(ax=axes[3,1])#, cmap='seismic')

#axes[0,0].set_ylim(-2, 5)
#axes[0,1].set_ylim(-2, 5)
#axes[1,0].set_ylim(-2, 5)
#axes[1,1].set_ylim(-2, 5)
#axes[2,0].set_ylim(-2, 5)
#axes[2,1].set_ylim(-2, 5)
#axes[3,0].set_ylim(-2, 5)
#axes[3,1].set_ylim(-2, 5)

fig.suptitle('Time-averaged tendency terms binned in WG region', fontsize=18, fontweight='bold')

axes[0,0].title.set_text('Horizontal Advective Flux Term')
axes[0,1].title.set_text('Horizontal Diffusive Flux Term')
axes[1,0].title.set_text('Vertical Advective Flux Term')
axes[1,1].title.set_text('Vertical Diffusive Flux Term')
axes[2,0].title.set_text('Surface Flux Term')
axes[2,1].title.set_text('Linear free-surface correction Term')
axes[3,0].title.set_text('Shortwave Term')
axes[3,1].title.set_text('Total Tendency Term')

plt.show() #this seems to work to make suptitle displayed neatly above
plt.tight_layout();

In [ ]:
plt.figure(figsize=(12,6))
ds_hist_wg_mean.to_array().plot.line(x='THETA_bin')
plt.title('Time-averaged tendency terms in\n full WG water column');

In [ ]:
ds_hist_wg_mean.tot_tend_hist_wg.plot(label='rhs')
ds_hist_wg_mean.tot_tend_truth_hist_wg.plot(label='lhs')
plt.legend();

## Interior WG column

In [ ]:
#index locations corresponding to above lat/lon coordinates
lower_lat_lower = 0
upper_lat_lower = 71
left_lon_lower = 1794
right_lon_lower = 2159

lower_cell_range = dict(Zl=[27, 42])
lower_cell_range_z = dict(Z=slice(27, 42))

In [ ]:
#set the boundaries
lat_range_lower = dict(YC=slice(lower_lat_lower, upper_lat_lower))
lon_range_lower = dict(XC=slice(left_lon_lower, right_lon_lower))

In [ ]:
ds_wg_lower = ds.isel(**lat_range_lower, **lon_range_lower, **lower_cell_range_z)
ds_wg_lower

In [ ]:
def hist_var(direction, tendency, suffix):
    """Compute histogram of `tendency` flux of tracer `suffix`
    where `tendency` is `ADV` or `DIFF`, `suffix` is `TH` or `SLT`,
    and `direction` is `horiz` or `vert`. Return a new xarray.Dataset."""
    
    var = 'conv_' + direction + '_' + tendency + '_flux_' + suffix
    hist_var = histogram(ds_wg_lower.THETA, bins=[theta_bins], weights=ds_wg_lower[var], dim=['XC', 'YC', 'Z'])
    return hist_var

In [ ]:
adv_vert_hist_wg_lower = hist_var('vert', 'adv', 'TH').load()
adv_horiz_hist_wg_lower = hist_var('horiz', 'adv', 'TH').load()
diff_vert_hist_wg_lower = hist_var('vert', 'diff', 'TH').load()
diff_horiz_hist_wg_lower = hist_var('horiz', 'diff', 'TH').load()

In [ ]:
total_tendency_truth_hist_wg_lower = histogram(ds_wg_lower.THETA, bins=[theta_bins], 
                                      weights=ds_wg_lower.total_tendency_TH_truth, 
                                      dim=['XC', 'YC', 'Z']).load()

In [ ]:
ds_hist_wg_lower = xr.merge([adv_horiz_hist_wg_lower.rename('adv_horiz_hist_wg_lower'), 
                             adv_vert_hist_wg_lower.rename('adv_vert_hist_wg_lower'),
                             diff_horiz_hist_wg_lower.rename('diff_horiz_hist_wg_lower'),
                             diff_vert_hist_wg_lower.rename('diff_vert_hist_wg_lower'),
                             total_tendency_truth_hist_wg_lower.rename('tot_tend_truth_hist_wg_lower')])
ds_hist_wg_lower

In [ ]:
total_tendency_hist_wg_lower = (ds_hist_wg_lower.adv_horiz_hist_wg_lower + 
                                ds_hist_wg_lower.adv_vert_hist_wg_lower + 
                                ds_hist_wg_lower.diff_horiz_hist_wg_lower + 
                                ds_hist_wg_lower.diff_vert_hist_wg_lower)

In [ ]:
ds_hist_wg_lower['tot_tend_hist_wg_lower'] = total_tendency_hist_wg_lower

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(14,14))

ds_hist_wg_lower.adv_horiz_hist_wg_lower.transpose().plot(ax=axes[0,0], cmap='seismic')
ds_hist_wg_lower.diff_horiz_hist_wg_lower.transpose().plot(ax=axes[0,1], cmap='seismic')
ds_hist_wg_lower.adv_vert_hist_wg_lower.transpose().plot(ax=axes[1,0], cmap='seismic')
ds_hist_wg_lower.diff_vert_hist_wg_lower.transpose().plot(ax=axes[1,1], cmap='seismic')
ds_hist_wg_lower.tot_tend_hist_wg_lower.transpose().plot(ax=axes[2,0], cmap='seismic')
ds_hist_wg_lower.tot_tend_truth_hist_wg_lower.transpose().plot(ax=axes[2,1], cmap='seismic')

axes[0,0].set_ylim(-2, 5)
axes[0,1].set_ylim(-2, 5)
axes[1,0].set_ylim(-2, 5)
axes[1,1].set_ylim(-2, 5)
axes[2,0].set_ylim(-2, 5)

fig.suptitle('Tendency terms binned in interior WG region', fontsize=18, fontweight='bold')

axes[0,0].title.set_text('Horizontal Advective Flux Term')
axes[0,1].title.set_text('Horizontal Diffusive Flux Term')
axes[1,0].title.set_text('Vertical Advective Flux Term')
axes[1,1].title.set_text('Vertical Diffusive Flux Term')
axes[2,0].title.set_text('Total Tendency Term')
axes[2,1].title.set_text('True Total Tendency Term')
#axes[2,1].title.set_text('Linear free-surface correction Term')
#axes[3,0].title.set_text('Shortwave Term')
#axes[3,1].title.set_text('Total Tendency Term')

plt.show() #this seems to work to make suptitle displayed neatly above
plt.tight_layout();

#### Time-Avged

In [ ]:
ds_hist_wg_lower_mean = ds_hist_wg_lower.mean(dim='time')
ds_hist_wg_lower_mean

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(14,14))

ds_hist_wg_lower_mean.adv_horiz_hist_wg_lower.transpose().plot(ax=axes[0,0])
ds_hist_wg_lower_mean.diff_horiz_hist_wg_lower.transpose().plot(ax=axes[0,1])
ds_hist_wg_lower_mean.adv_vert_hist_wg_lower.transpose().plot(ax=axes[1,0])
ds_hist_wg_lower_mean.diff_vert_hist_wg_lower.transpose().plot(ax=axes[1,1])
ds_hist_wg_lower_mean.total_tendency_hist_wg_lower.transpose().plot(ax=axes[2,0])

fig.suptitle('Tendency terms binned in interior WG region', fontsize=18, fontweight='bold')

axes[0,0].title.set_text('Horizontal Advective Flux Term')
axes[0,1].title.set_text('Horizontal Diffusive Flux Term')
axes[1,0].title.set_text('Vertical Advective Flux Term')
axes[1,1].title.set_text('Vertical Diffusive Flux Term')
axes[2,0].title.set_text('Total Tendency Term')
#axes[2,1].title.set_text('Linear free-surface correction Term')
#axes[3,0].title.set_text('Shortwave Term')
#axes[3,1].title.set_text('Total Tendency Term')

plt.show() #this seems to work to make suptitle displayed neatly above
plt.tight_layout();

In [ ]:
plt.figure(figsize=(12,6))
ds_hist_wg_lower_mean.to_array().plot.line(x='THETA_bin')
plt.title('Time-averaged tendency terms in\n interior WG water column');

In [ ]:
plt.figure(figsize=(12,6))
ds_hist_wg_lower_mean.tot_tend_hist_wg_lower.plot(label='rhs')
ds_hist_wg_lower_mean.tot_tend_truth_hist_wg_lower.plot(label='lhs')
plt.legend();

In [ ]:
sst = ds.THETA.isel(Z=0)
sss = ds.SALT.isel(Z=0)

In [ ]:
#converting practical salinity to absolute salinity
sa = xr.apply_ufunc(gsw.SA_from_SP, sss, 0, ds.XC, ds.YC,
                    output_dtypes=[sss.dtype],
                    dask='parallelized').reset_coords(drop=True)
sa

## Temperature component of global WMT budget

In [ ]:
#vertical flux
#ds.TFLUX.isel(**lat_range, **lon_range)
t_flux_z = (ds.TFLUX * coords.rA)#.sum(dim=['XC','YC'])
t_flux_z
#units: W = (kg*m^2)/s^3

In [ ]:
tflux_hist = histogram(sst, bins=[theta_bins], weights=t_flux_z, dim=['XC', 'YC'])
tflux_hist

In [ ]:
tflux_hist.load()

In [ ]:
tflux_hist.transpose().plot()
plt.ylim([-2, 10])

In [ ]:
sst_hist = histogram(sst, bins=[theta_bins], dim=['XC', 'YC'])
sst_hist.load()

In [ ]:
sst_hist.transpose().plot()
plt.ylim([-2, 10])

In [ ]:
sst[0].isnull().plot()

In [ ]:
sst_hist.mean(dim='time').plot()

In [ ]:
surface_weights = coords.rA*coords.hFacC[0]

In [ ]:
sst_hist = histogram(sst, bins=[theta_bins], weights=surface_weights, dim=['XC', 'YC'])
sst_hist.load()

In [ ]:
(sst_hist.mean(dim='time')/surface_weights.sum()).plot()

In [ ]:
# volume tendency term
vol_c = coords.hFacC * coords.rA * coords.drF
theta_vol = ds.THETA

# define bins
delta_theta = 0.1
theta_bins = np.arange(-2, 20, delta_theta)

theta_dist = histogram(ds.THETA, bins=[theta_bins],
                       weights=vol_c, dim=['XC', 'YC', 'Z'])
theta_dist

In [ ]:
theta_dist.load()

In [ ]:
# most of the world's ocean is ~2˚C
# very little variability in temp distribution
theta_dist.plot(x='time')

In [ ]:
# most of the world's ocean is cold (centered ~2˚C)
theta_dist.mean(dim='time').plot()

In [ ]:
# thets distribution anomaly
(theta_dist - theta_dist.mean(dim='time')).plot(x='time')
plt.ylim([-2, 5])

In [ ]:
# cumulative volume
V = theta_dist.cumsum(dim='THETA_bin')
V.plot()

In [ ]:
# avg of cumulative volume
V_mean = V.mean(dim='time')
V_mean.plot()

In [ ]:
# cumulative volume anomaly
V_anom = V - V_mean
V_anom.plot(x='time')
plt.ylim(-2, 7)

In [ ]:
delta_t = 5*24*60*60
dVdt = V_anom.diff(dim='time') / delta_t
#dV/dt
dVdt.plot(x='time', vmax=200e6)
plt.ylim(-2, 7)

In [ ]:
(dVdt.mean(dim='time') / 1e6).plot()

In [ ]:
# one WMT term

runit2mass = 1.035e3 #rho_not
heat_capacity_cp = 3.994e3 #J/˚K/kg
Omega_tflux = tflux_hist / (runit2mass * heat_capacity_cp) / delta_theta

Omega_tflux.plot(x='time', vmax=200e6)

In [ ]:
# psi - side flux term
V_volflux = ds.VVEL.isel(YG=-1) * coords.hFacS.isel(YG=-1) * coords.drF * coords.dxG.isel(YG=-1)
V_volflux

In [ ]:
import xgcm
grid = xgcm.Grid(coords, periodic=['X'])
grid

In [ ]:
theta_s = grid.interp(ds.THETA, 'Y', boundary='extrapolate')
theta_s

In [ ]:
theta_s.data

In [ ]:
# just use cell center for binning transport (rather than interpolating to cell face)
Theta_nb = ds.THETA.isel(YC=-1).reset_coords(drop=True)
Theta_nb

In [ ]:
V_binned = histogram(Theta_nb, bins=[theta_bins], weights=V_volflux, dim=['XC', 'Z'])
V_binned

In [ ]:
V_binned.load()

In [ ]:
V_binned.plot(x='time')

In [ ]:
V_binned.mean(dim='time').plot()

## Temperature component of WG WMT budget

In [ ]:
sst_mean = ds.THETA.isel(Z=0).mean(dim='time')#.load()

In [ ]:
surface_mask = (coords.hFacC[0]>0)#.load()

In [ ]:
sst_mean.where(surface_mask).plot()

In [ ]:
left_lon = 298
upper_lat = -65
right_lon = 360
lower_lat = -78

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), subplot_kw={'facecolor': '0.5'})
(sst_mean.where(surface_mask)).plot(ax=ax, vmin=-2, vmax=10)
ax.plot([left_lon, right_lon], [upper_lat, upper_lat], color='w')
ax.plot([right_lon, right_lon], [lower_lat, upper_lat], color='w')
ax.plot([left_lon, left_lon], [lower_lat, upper_lat], color='w')
ax.plot([left_lon, right_lon], [lower_lat, lower_lat], color='w')
#ax.patch(xy=(-80, 290), width=70, height=20)
#ax.patch(xy=(295,-80), width = 1, height = 1)
ax.set_xlim([287, 362])
ax.set_ylim([-80, -55])
plt.title('Surface Mask @ ~2km\nLat[290,360]\nLon[-80,-63.5]');

In [ ]:
lower_lat = 0
upper_lat = 78
left_lon = 1788
right_lon = 2159

In [ ]:
#set the boundaries
lat_range = dict(YC=slice(lower_lat, upper_lat))
lon_range = dict(XC=slice(left_lon, right_lon))

In [ ]:
sst_wg = ds.THETA.isel(**lat_range, **lon_range, Z=0)
sst_wg

In [ ]:
#vertical flux
t_flux_z_wg = (ds.TFLUX.isel(**lat_range, **lon_range) * coords.rA)
t_flux_z_wg
#units: W = (kg*m^2)/s^3

In [ ]:
tflux_hist_wg = histogram(sst_wg, bins=[theta_bins], 
                          weights=t_flux_z_wg, dim=['XC', 'YC'])
tflux_hist_wg

In [ ]:
tflux_hist_wg.load()

In [ ]:
tflux_hist_wg.transpose().plot()
plt.ylim([-2, 10])

In [ ]:
sst_hist_wg = histogram(sst_wg, bins=[theta_bins], dim=['XC', 'YC'])
sst_hist_wg.load()

In [ ]:
sst_hist_wg.transpose().plot()
plt.ylim([-2, 10])

In [ ]:
#WG surface water is mainly made up of waters of -1˚C and -0.2˚C
sst_hist_wg.mean(dim='time').plot()
plt.xlim(-2, 2)

In [ ]:
surface_weights_wg = (coords.rA.isel(**lat_range, **lon_range) *
                   coords.hFacC[0].isel(**lat_range, **lon_range))

In [ ]:
sst_hist_wg_surfweights = histogram(sst_wg, bins=[theta_bins], 
                                    weights=surface_weights_wg, dim=['XC', 'YC'])
sst_hist_wg_surfweights.load()

In [ ]:
(sst_hist_wg_surfweights.mean(dim='time')/surface_weights_wg.sum()).plot()
plt.xlim(-2,-1.3)

In [ ]:
# volume tendency term in WG
vol_c_wg = (coords.hFacC * coords.rA * coords.drF).isel(**lat_range, **lon_range)
theta_vol_wg = ds.THETA

# theta distribution in WG
theta_dist_wg = histogram(ds.THETA.isel(**lat_range, **lon_range), bins=[theta_bins],
                       weights=vol_c_wg, dim=['XC', 'YC', 'Z'])
theta_dist_wg

In [ ]:
theta_dist_wg.load()

In [ ]:
# most of the wg ocean is between -1.0˚C < wg < 1.0˚C
# little variability in temp distribution
theta_dist_wg.plot(x='time')
plt.ylim(-2, 2)

In [ ]:
# most of the wg ocean is cold (centered ~-0.3˚C)
theta_dist_wg.mean(dim='time').plot()
plt.xlim(-2, 2)

In [ ]:
# theta distribution anomaly
(theta_dist_wg - theta_dist_wg.mean(dim='time')).plot(x='time')
plt.ylim([-2, 1.5])

In [ ]:
# cumulative volume
V_wg = theta_dist_wg.cumsum(dim='THETA_bin')
V_wg.transpose().plot()

In [ ]:
# avg of cumulative volume
V_wg_mean = V_wg.mean(dim='time')
V_wg_mean.plot()

In [ ]:
# cumulative volume anomaly
V_wg_anom = V_wg - V_wg_mean
V_wg_anom.plot(x='time', vmax=6e14)
plt.ylim(-2, 7)

In [ ]:
delta_t = 5*24*60*60 #in nanoseconds like the dataset time
dVdt_wg = V_wg_anom.diff(dim='time') / delta_t

dVdt_wg.plot(x='time', vmin=-2e8, vmax=2e8)
plt.ylim(-2, 7)

In [ ]:
#does this mean the main watermass being transformed has temperature of 0˚C
#or does it mean that most water is being transformed to this property
(dVdt_wg.mean(dim='time') / 1e6).plot() # in SV

### diff term

Want to calculate convergence of horizontal diffusive fluxes

$$ - \frac{\partial}{\partial x} F^x_{diff} - \frac{\partial}{\partial y} F^y_{diff} $$

In [ ]:
lat_range = dict(YC=slice(lower_lat, upper_lat))
lon_range = dict(XC=slice(left_lon, right_lon))

ds_wg = ds.isel(YC=slice(lower_lat, upper_lat), YG=slice(lower_lat, upper_lat),
                XC=slice(left_lon, right_lon), XG=slice(left_lon, right_lon))

import xgcm
grid_wg = xgcm.Grid(ds_wg, periodic=False)
grid_wg

In [ ]:
horizontal_diffusive_heat_flux_convergence = (
    - grid_wg.diff(ds_wg.DFyE_TH, 'Y', boundary='fill')
    - grid_wg.diff(ds_wg.DFxE_TH, 'X', boundary='fill') )

horizontal_diffusive_heat_flux_convergence = \
  horizontal_diffusive_heat_flux_convergence.chunk({'YC': ds_wg.dims['YC'],
                                                    'XC': ds_wg.dims['XC']})

horizontal_diffusive_heat_flux_convergence

In [ ]:
horizontal_diffusive_heat_flux_convergence.data

In [ ]:
horizontal_diffusive_flux_wmt_term = \
   histogram(ds_wg.THETA,
             bins=[theta_bins],
             weights=horizontal_diffusive_heat_flux_convergence,
             dim=['XC', 'YC', 'Z'])

horizontal_diffusive_flux_wmt_term

In [ ]:
horizontal_diffusive_flux_wmt_term.load()

In [ ]:
horizontal_diffusive_flux_wmt_term.plot(x='time', vmax=1e6)
plt.ylim([-2, 2])

In [ ]:

ds_wg.DFyE_TH.diff('YG')

In [ ]:
diff_flux.nbytes / 1e9

In [ ]:
#diffusive flux
#don't sum over dimensions because you need to bin later
diff_flux_y = ds.DFyE_TH.isel(**lon_range, YG=upper_lat)
diff_flux_x = ds.DFxE_TH.isel(**lat_range, XG=right_lon)
print(diff_flux_y.dims)
print(diff_flux_x.dims)

In [ ]:
diff_flux = diff_flux_x + diff_flux_y
diff_flux.load()
#units: (˚C m^3)/s

In [ ]:
vol_c_wg

In [ ]:
diff_dist = histogram(diff_flux, bins=[theta_bins], weights=vol_c_wg, dim=['XG', 'YG', 'Z'])

In [ ]:
diff_dist = histogram(, bins=[teht])

In [ ]:
diff_dist.load()

In [ ]:
# most of the world's ocean is ~2˚C
# very little variability in temp distribution
diff_dist.plot(x='time')

In [ ]:
# most of the world's ocean is cold (centered ~2˚C)
diff_dist.mean(dim='time').plot()

In [ ]:
# thets distribution anomaly
(diff_dist - diff_dist.mean(dim='time')).plot(x='time')
#plt.ylim([-2, 5])